# Large-Scale E-commerce Data Cleaning & Standardization

## Problem Statement

This notebook demonstrates a **production-grade data cleaning pipeline** applied to a
125,000-row messy e-commerce export. The dataset contains **8 distinct data quality issues**
that are typical of real-world exports from platforms like Shopify, WooCommerce, and
custom ERP systems.

> **Note:** Simulated large-scale e-commerce export for demonstration purposes.

### Data Quality Issues Addressed

1. **Duplicate rows** — ~5,000 exact duplicates injected into the dataset
2. **Mixed date formats** — 5 different date representations (US, EU, ISO, etc.)
3. **Currency formatting** — Prices with `$`, `€`, `USD`, `TL` prefixes/suffixes and European decimal commas
4. **SKU casing inconsistencies** — Mixed upper/lower case and missing dashes (`SKU001` vs `SKU-001`)
5. **Missing values with multiple null representations** — `N/A`, `n/a`, `-`, `none`, `null`, `""`, etc.
6. **Encoding corruption (mojibake)** — UTF-8 product names decoded as Latin-1 producing artifacts like `Ã¼`
7. **Inconsistent country names** — Free-text entries like `us`, `USA`, `United States` for the same country
8. **Status typos & casing** — Misspellings like `deliverred`, `Cancellled` plus mixed casing

---
## Section 2 — Initial Exploration

In [1]:
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 140)

In [2]:
df_raw = pd.read_csv("messy_ecommerce_export.csv")
print(f"Dataset shape: {df_raw.shape[0]:,} rows x {df_raw.shape[1]} columns")

Dataset shape: 125,000 rows x 13 columns


In [3]:
print("Column data types:\n")
print(df_raw.dtypes)

Column data types:

order_id             object
sku                  object
product_name         object
order_date           object
price                object
quantity            float64
customer_email       object
customer_phone       object
shipping_country     object
status               object
_unnamed_1          float64
_unnamed_2          float64
notes               float64
dtype: object


In [4]:
print("First 10 rows:\n")
df_raw.head(10)

First 10 rows:



,order_id,sku,product_name,order_date,price,quantity,customer_email,customer_phone,shipping_country,status,_unnamed_1,_unnamed_2,notes
0,ORD-128360,sku-005,NaÃ¯ve Art Print Set,"May 28, 2023",62.88,11.0,NaN,NaN,GB,Cancellled,NaN,NaN,NaN
1,ORD-122200,SKU003,ÃlÃ§Ã¼ Aleti Premium,2023-01-26,"€141,43",2.0,NaN,+1-555-6983,US,Processing,NaN,NaN,NaN
2,ORD-42132,sku-002,TÃ¼rkÃ§e Klavye Seti,14.04.2023,256.01 TL,3.0,dave.brown@hotmail.com,555.819.8173,germany,cancelled,NaN,NaN,NaN
3,ORD-120443,SKU007,CrÃ¨me BrÃ»lÃ©e Torch Kit,"Dec 24, 2024","€37,42",2.0,bob.smith@yahoo.com,5554475,Australia,Pending,NaN,NaN,NaN
4,ORD-81732,SKU004,Café Blend Dark Roast,06/06/2023,27.43 TL,4.0,alice.johnson@gmail.com,NaN,germany,delivered,NaN,NaN,NaN
5,ORD-112797,Sku-010,El Niño Weather Station,05.09.2024,$11.78,18.0,NaN,5556970,canada,shipped,NaN,NaN,NaN
6,ORD-70861,SKU008,PiÃ±ata Party Pack,2023-04-14,"€128,42",17.0,NaN,+1-555-9284,GB,delivered,NaN,NaN,NaN
7,ORD-14473,Sku-008,Piñata Party Pack,21.10.2024,59.29 TL,4.0,NaN,5554717,AU,Pending,NaN,NaN,NaN
8,ORD-126301,SKU008,Piñata Party Pack,16-11-2023,73.82 TL,6.0,frank.miller@icloud.com,+1-555-5391,CA,processing,NaN,NaN,NaN
9,ORD-54576,SKU003,Ölçü Aleti Premium,2024-12-12,195.58,8.0,NaN,(555) 737-3631,GB,cancelled,NaN,NaN,NaN


In [5]:
null_summary = pd.DataFrame({
    "Null Count": df_raw.isnull().sum(),
    "Null %": (df_raw.isnull().sum() / len(df_raw) * 100).round(2),
})
print("Null summary:\n")
print(null_summary)

Null summary:

                  Null Count  Null %
order_id                   0    0.00
sku                        0    0.00
product_name               0    0.00
order_date                 0    0.00
price                      0    0.00
quantity                   0    0.00
customer_email         30329   24.26
customer_phone         43654   34.92
shipping_country           0    0.00
status                     0    0.00
_unnamed_1            125000  100.00
_unnamed_2            125000  100.00
notes                 125000  100.00


In [6]:
dup_count = df_raw.duplicated().sum()
print(f"Duplicate rows: {dup_count:,}")

Duplicate rows: 5,000


In [7]:
# Save before-cleaning snapshots for later comparison
before_shape = df_raw.shape
before_dtypes = df_raw.dtypes.copy()
before_nulls = df_raw.isnull().sum().sum()
before_sample = df_raw.head(6).copy()

# Work on a copy from here on
df = df_raw.copy()
print("Snapshots saved. Working copy created.")

Snapshots saved. Working copy created.


---
## Section 3 — Cleaning Pipeline (10 Steps)

Each step is an isolated, timed cell so performance can be monitored independently.

### Step 1 — Drop Empty Columns
Remove columns where **every** value is null.

In [8]:
%%time
empty_cols = [col for col in df.columns if df[col].isnull().all()]
print(f"Empty columns found: {empty_cols}")
df.drop(columns=empty_cols, inplace=True)
print(f"Columns after drop: {list(df.columns)}")
print(f"Shape: {df.shape}")

Empty columns found: ['_unnamed_1', '_unnamed_2', 'notes']
Columns after drop: ['order_id', 'sku', 'product_name', 'order_date', 'price', 'quantity', 'customer_email', 'customer_phone', 'shipping_country', 'status']
Shape: (125000, 10)
CPU times: total: 31.2 ms
Wall time: 26.6 ms


### Step 2 — Remove Duplicate Rows

In [9]:
%%time
rows_before = len(df)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
rows_after = len(df)
dups_removed = rows_before - rows_after
print(f"Rows before: {rows_before:,}")
print(f"Rows after:  {rows_after:,}")
print(f"Duplicates removed: {dups_removed:,}")

Rows before: 125,000


Rows after:  120,000
Duplicates removed: 5,000
CPU times: total: 62.5 ms
Wall time: 68.5 ms


### Step 3 — Standardize Dates to ISO 8601 (`YYYY-MM-DD`)

In [10]:
%%time
DATE_FORMATS = ["%m/%d/%Y", "%Y-%m-%d", "%d-%m-%Y", "%b %d, %Y", "%d.%m.%Y"]

def parse_date(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip().strip('"').strip("'")
    for fmt in DATE_FORMATS:
        try:
            return pd.to_datetime(val, format=fmt).strftime("%Y-%m-%d")
        except (ValueError, TypeError):
            continue
    return np.nan

sample_before = df["order_date"].head(8).tolist()
df["order_date"] = df["order_date"].apply(parse_date)
sample_after = df["order_date"].head(8).tolist()

print("Date conversion sample:")
for b, a in zip(sample_before, sample_after):
    print(f"  {str(b):25s} -> {a}")
print(f"\nNull dates after conversion: {df['order_date'].isnull().sum()}")

Date conversion sample:
  May 28, 2023              -> 2023-05-28
  2023-01-26                -> 2023-01-26
  14.04.2023                -> 2023-04-14
  Dec 24, 2024              -> 2024-12-24
  06/06/2023                -> 2023-06-06
  05.09.2024                -> 2024-09-05
  2023-04-14                -> 2023-04-14
  21.10.2024                -> 2024-10-21

Null dates after conversion: 0
CPU times: total: 7.28 s
Wall time: 7.32 s


### Step 4 — Normalize Prices to Float

In [11]:
%%time
def clean_price(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    # Remove currency symbols and words
    s = s.replace("$", "").replace("USD", "").replace("TL", "").replace("\u20ac", "").strip()
    # Handle European format: €1.234,56 -> 1234.56
    # If both . and , exist and , is after ., it's European
    if "," in s and "." in s:
        if s.rfind(",") > s.rfind("."):
            # European: 1.234,56
            s = s.replace(".", "").replace(",", ".")
        else:
            # US: 1,234.56
            s = s.replace(",", "")
    elif "," in s and "." not in s:
        # Could be European decimal: 19,99
        s = s.replace(",", ".")
    try:
        return round(float(s), 2)
    except ValueError:
        return np.nan

df["price"] = df["price"].apply(clean_price)
print(f"Price range: ${df['price'].min():.2f} — ${df['price'].max():.2f}")
print(f"Null prices: {df['price'].isnull().sum()}")
print(f"Mean price:  ${df['price'].mean():.2f}")

Price range: $4.99 — $299.99
Null prices: 0
Mean price:  $152.65
CPU times: total: 109 ms
Wall time: 96.3 ms


### Step 5 — Standardize SKU Format
Uppercase all SKUs and ensure `SKU-XXX` format with dash.

In [12]:
%%time
def clean_sku(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip().upper()
    # Insert dash if missing: SKU001 -> SKU-001
    if s.startswith("SKU") and len(s) > 3 and s[3] != "-":
        s = "SKU-" + s[3:]
    return s

df["sku"] = df["sku"].apply(clean_sku)
print(f"Unique SKUs after standardization: {df['sku'].nunique()}")
print(f"SKU values: {sorted(df['sku'].unique())}")

Unique SKUs after standardization: 10
SKU values: ['SKU-001', 'SKU-002', 'SKU-003', 'SKU-004', 'SKU-005', 'SKU-006', 'SKU-007', 'SKU-008', 'SKU-009', 'SKU-010']
CPU times: total: 46.9 ms
Wall time: 48 ms


### Step 6 — Normalize Missing Values
Convert fake null representations to `np.nan` in email and phone columns.

In [13]:
%%time
FAKE_NULLS = {"N/A", "n/a", "na", "NA", "-", "--", ".", "none", "None", "null", "NULL", ""}

def coerce_null(val):
    if pd.isna(val):
        return np.nan
    if str(val).strip() in FAKE_NULLS:
        return np.nan
    return val

for col in ["customer_email", "customer_phone"]:
    before_nulls = df[col].isnull().sum()
    df[col] = df[col].apply(coerce_null)
    after_nulls = df[col].isnull().sum()
    coerced = after_nulls - before_nulls
    print(f"{col}: coerced {coerced:,} fake nulls -> np.nan  (total null now: {after_nulls:,})")

customer_email: coerced 7,216 fake nulls -> np.nan  (total null now: 36,343)


customer_phone: coerced 0 fake nulls -> np.nan  (total null now: 41,914)
CPU times: total: 62.5 ms
Wall time: 62.7 ms


### Step 7 — Fix Encoding Corruption (Mojibake)
Product names were corrupted by a UTF-8 → Latin-1 encoding mismatch,
producing artifacts like `Ã¼` instead of `ü`.

In [14]:
%%time
# Mojibake pattern: when UTF-8 bytes are misread as Latin-1, you get sequences
# like \xc3\x83, \xc3\xbc rendered as Ã¼, Ã¶, etc.
MOJIBAKE_RE = re.compile(r"\xc3[\x80-\xbf]")

def fix_encoding(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    try:
        fixed = s.encode("latin-1").decode("utf-8")
        return fixed.strip()
    except (UnicodeDecodeError, UnicodeEncodeError):
        return s.strip()

# Count corrupted rows before fix (detect mojibake double-byte artifacts)
corrupted_before = df["product_name"].apply(
    lambda x: bool(MOJIBAKE_RE.search(str(x))) if pd.notna(x) else False
).sum()

df["product_name"] = df["product_name"].apply(fix_encoding)

corrupted_after = df["product_name"].apply(
    lambda x: bool(MOJIBAKE_RE.search(str(x))) if pd.notna(x) else False
).sum()

print(f"Corrupted product names before fix: {corrupted_before:,}")
print(f"Corrupted product names after fix:  {corrupted_after:,}")
print(f"Unique product names: {sorted(df['product_name'].dropna().unique())}")

Corrupted product names before fix: 36,451


Corrupted product names after fix:  0
Unique product names: ['Café Blend Dark Roast', 'Crème Brûlée Torch Kit', 'El Niño Weather Station', 'Grüner Tee', 'Naïve Art Print Set', 'Piñata Party Pack', 'Résumé Template Pro', 'Türkçe Klavye Seti', 'Ölçü Aleti Premium', 'Über Comfort Pillow']
CPU times: total: 156 ms
Wall time: 174 ms


### Step 8 — Standardize Phone Numbers
Extract digits and reformat to `XXX-XXX-XXXX`.

In [15]:
%%time
def clean_phone(val):
    if pd.isna(val):
        return np.nan
    digits = re.sub(r"\D", "", str(val))
    if len(digits) == 0:
        return np.nan
    # 7 digits: 555XXXX -> 555-XXX-XXXX (prepend area code)
    if len(digits) == 7:
        return f"555-{digits[:3]}-{digits[3:]}"
    # 8 digits starting with 1: country code + 555 + XXXX -> 555-XXX-XXXX
    elif len(digits) == 8 and digits[0] == "1":
        digits = digits[1:]  # strip country code -> 555XXXX
        return f"555-{digits[:3]}-{digits[3:]}"
    # 10 digits: XXX-XXX-XXXX
    elif len(digits) == 10:
        return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"
    # 11 digits starting with 1: strip country code -> 10 digits
    elif len(digits) == 11 and digits[0] == "1":
        digits = digits[1:]
        return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"
    else:
        return np.nan

df["customer_phone"] = df["customer_phone"].apply(clean_phone)
sample_phones = df["customer_phone"].dropna().head(8).tolist()
print("Phone number sample after cleaning:")
for p in sample_phones:
    print(f"  {p}")
print(f"\nNull phones: {df['customer_phone'].isnull().sum():,}")

Phone number sample after cleaning:
  555-555-6983
  555-819-8173
  555-555-4475
  555-555-6970
  555-555-9284
  555-555-4717
  555-555-5391
  555-737-3631

Null phones: 41,914
CPU times: total: 93.8 ms
Wall time: 97.3 ms


### Step 9 — Normalize Country Names to ISO Codes

In [16]:
%%time
COUNTRY_MAP = {
    "us": "US", "usa": "US", "united states": "US",
    "ca": "CA", "canada": "CA",
    "gb": "GB", "uk": "GB", "united kingdom": "GB",
    "de": "DE", "germany": "DE",
    "fr": "FR", "france": "FR",
    "au": "AU", "australia": "AU",
}

def clean_country(val):
    if pd.isna(val):
        return np.nan
    key = str(val).strip().lower()
    return COUNTRY_MAP.get(key, val)

df["shipping_country"] = df["shipping_country"].apply(clean_country)
print("Country value counts after normalization:\n")
print(df["shipping_country"].value_counts())

Country value counts after normalization:

shipping_country
US    28336
DE    21357
GB    21210
CA    21060
FR    14056
AU    13981
Name: count, dtype: int64
CPU times: total: 31.2 ms
Wall time: 36.4 ms


### Step 10 — Fix Status Typos & Casing

In [17]:
%%time
STATUS_TYPO_MAP = {
    "deliverred": "delivered",
    "cancellled": "cancelled",
}

def clean_status(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip().lower()
    return STATUS_TYPO_MAP.get(s, s)

df["status"] = df["status"].apply(clean_status)
print("Status value counts after cleaning:\n")
print(df["status"].value_counts())

Status value counts after cleaning:



status
delivered     29929
shipped       29879
cancelled     20202
processing    20153
pending       19837
Name: count, dtype: int64
CPU times: total: 46.9 ms
Wall time: 38.7 ms


---
## Section 4 — Validation & Summary

### Cleaning Summary Table

In [18]:
# Dynamically compute summary from actual data
after_shape = df.shape

# Compute metrics from the before/after snapshots
empty_cols_dropped = before_shape[1] - after_shape[1]  # columns removed
dups_removed_count = before_shape[0] - len(df)  # approximate (includes dup removal)

summary_data = [
    ["1. Drop Empty Columns", f"{empty_cols_dropped} columns",
     "Removed columns where 100% of values were null"],
    ["2. Remove Duplicates", f"{before_shape[0] - len(df_raw.drop_duplicates()):,} rows",
     "Dropped exact duplicate rows"],
    ["3. Standardize Dates", f"{df_raw['order_date'].nunique():,} unique formats",
     "Parsed 5 date formats into ISO 8601 (YYYY-MM-DD)"],
    ["4. Normalize Prices", f"{df_raw['price'].nunique():,} unique formats",
     "Stripped currency symbols, fixed decimal separators, converted to float"],
    ["5. Standardize SKUs", f"{df_raw['sku'].nunique():,} variants -> {df['sku'].nunique()}",
     "Uppercased and ensured SKU-XXX dash format"],
    ["6. Normalize Missing Values",
     f"{(df_raw['customer_email'].isin(['N/A','n/a','na','NA','-','--','.','none','None','null','NULL',''])).sum() + (df_raw['customer_phone'].isin(['N/A','n/a','na','NA','-','--','.','none','None','null','NULL',''])).sum():,} fake nulls",
     "Converted N/A, none, null, -, etc. to np.nan"],
    ["7. Fix Encoding", f"{corrupted_before:,} rows",
     "Repaired Latin-1 mojibake in product names"],
    ["8. Standardize Phones",
     f"{df['customer_phone'].notna().sum():,} formatted",
     "Extracted digits, reformatted to XXX-XXX-XXXX"],
    ["9. Normalize Countries",
     f"{df_raw['shipping_country'].nunique()} variants -> {df['shipping_country'].nunique()}",
     "Mapped free-text country names to ISO 2-letter codes"],
    ["10. Fix Status Typos",
     f"{df_raw['status'].nunique()} variants -> {df['status'].nunique()}",
     "Corrected typos (deliverred, cancellled) and lowercased"],
]

summary_df = pd.DataFrame(summary_data, columns=["Issue", "Rows Affected", "Action Taken"])
summary_df.style.set_properties(**{"text-align": "left"}).hide(axis="index")

Issue,Rows Affected,Action Taken
1. Drop Empty Columns,3 columns,Removed columns where 100% of values were null
2. Remove Duplicates,"5,000 rows",Dropped exact duplicate rows
3. Standardize Dates,"3,650 unique formats",Parsed 5 date formats into ISO 8601 (YYYY-MM-DD)
4. Normalize Prices,"81,995 unique formats","Stripped currency symbols, fixed decimal separators, converted to float"
5. Standardize SKUs,40 variants -> 10,Uppercased and ensured SKU-XXX dash format
6. Normalize Missing Values,"7,549 fake nulls","Converted N/A, none, null, -, etc. to np.nan"
7. Fix Encoding,"36,451 rows",Repaired Latin-1 mojibake in product names
8. Standardize Phones,"78,086 formatted","Extracted digits, reformatted to XXX-XXX-XXXX"
9. Normalize Countries,17 variants -> 6,Mapped free-text country names to ISO 2-letter codes
10. Fix Status Typos,12 variants -> 5,"Corrected typos (deliverred, cancellled) and lowercased"


### Data Type Audit — Before vs After

In [19]:
after_dtypes = df.dtypes

# Build comparison for columns that exist in both
common_cols = [c for c in before_dtypes.index if c in after_dtypes.index]
dtype_comparison = pd.DataFrame({
    "Column": common_cols,
    "Before": [str(before_dtypes[c]) for c in common_cols],
    "After": [str(after_dtypes[c]) for c in common_cols],
})
dtype_comparison["Changed"] = dtype_comparison["Before"] != dtype_comparison["After"]

# Dropped columns
dropped_cols = [c for c in before_dtypes.index if c not in after_dtypes.index]
if dropped_cols:
    for dc in dropped_cols:
        dtype_comparison = pd.concat([dtype_comparison, pd.DataFrame([{
            "Column": dc, "Before": str(before_dtypes[dc]), "After": "(dropped)", "Changed": True
        }])], ignore_index=True)

def highlight_changed(row):
    if row["Changed"]:
        return ["background-color: #d4edda"] * len(row)
    return [""] * len(row)

dtype_comparison.style.apply(highlight_changed, axis=1).hide(axis="index")

Column,Before,After,Changed
order_id,object,object,False
sku,object,object,False
product_name,object,object,False
order_date,object,object,False
price,object,float64,True
quantity,float64,float64,False
customer_email,object,object,False
customer_phone,object,object,False
shipping_country,object,object,False
status,object,object,False


### Schema Comparison — Before vs After

In [20]:
before_dup_count = df_raw.duplicated().sum()
after_dup_count = df.duplicated().sum()
before_numeric = df_raw.select_dtypes(include="number").shape[1]
after_numeric = df.select_dtypes(include="number").shape[1]

schema_data = {
    "Metric": ["Rows", "Columns", "Total null cells", "Duplicate rows", "Numeric columns"],
    "Before": [
        f"{before_shape[0]:,}",
        str(before_shape[1]),
        f"{before_nulls:,}",
        f"{before_dup_count:,}",
        str(before_numeric),
    ],
    "After": [
        f"{after_shape[0]:,}",
        str(after_shape[1]),
        f"{df.isnull().sum().sum():,}",
        f"{after_dup_count:,}",
        str(after_numeric),
    ],
}
schema_df = pd.DataFrame(schema_data)
schema_df.style.set_properties(**{"text-align": "left"}).hide(axis="index")

Metric,Before,After
Rows,"125,000","120,000"
Columns,13,10
Total null cells,"41,914","78,257"
Duplicate rows,"5,000",0
Numeric columns,4,2


### Before / After Sample (First 6 Rows)

In [21]:
print("=== BEFORE CLEANING ===\n")
display(before_sample)
print("\n=== AFTER CLEANING ===\n")
display(df.head(6))

=== BEFORE CLEANING ===



,order_id,sku,product_name,order_date,price,quantity,customer_email,customer_phone,shipping_country,status,_unnamed_1,_unnamed_2,notes
0,ORD-128360,sku-005,NaÃ¯ve Art Print Set,"May 28, 2023",62.88,11.0,NaN,NaN,GB,Cancellled,NaN,NaN,NaN
1,ORD-122200,SKU003,ÃlÃ§Ã¼ Aleti Premium,2023-01-26,"€141,43",2.0,NaN,+1-555-6983,US,Processing,NaN,NaN,NaN
2,ORD-42132,sku-002,TÃ¼rkÃ§e Klavye Seti,14.04.2023,256.01 TL,3.0,dave.brown@hotmail.com,555.819.8173,germany,cancelled,NaN,NaN,NaN
3,ORD-120443,SKU007,CrÃ¨me BrÃ»lÃ©e Torch Kit,"Dec 24, 2024","€37,42",2.0,bob.smith@yahoo.com,5554475,Australia,Pending,NaN,NaN,NaN
4,ORD-81732,SKU004,Café Blend Dark Roast,06/06/2023,27.43 TL,4.0,alice.johnson@gmail.com,NaN,germany,delivered,NaN,NaN,NaN
5,ORD-112797,Sku-010,El Niño Weather Station,05.09.2024,$11.78,18.0,NaN,5556970,canada,shipped,NaN,NaN,NaN



=== AFTER CLEANING ===



,order_id,sku,product_name,order_date,price,quantity,customer_email,customer_phone,shipping_country,status
0,ORD-128360,SKU-005,Naïve Art Print Set,2023-05-28,62.88,11.0,NaN,NaN,GB,cancelled
1,ORD-122200,SKU-003,Ölçü Aleti Premium,2023-01-26,141.43,2.0,NaN,555-555-6983,US,processing
2,ORD-42132,SKU-002,Türkçe Klavye Seti,2023-04-14,256.01,3.0,dave.brown@hotmail.com,555-819-8173,DE,cancelled
3,ORD-120443,SKU-007,Crème Brûlée Torch Kit,2024-12-24,37.42,2.0,bob.smith@yahoo.com,555-555-4475,AU,pending
4,ORD-81732,SKU-004,Café Blend Dark Roast,2023-06-06,27.43,4.0,alice.johnson@gmail.com,NaN,DE,delivered
5,ORD-112797,SKU-010,El Niño Weather Station,2024-09-05,11.78,18.0,NaN,555-555-6970,CA,shipped


### Performance & Automation Notes

In [22]:
print(f"Dataset dimensions BEFORE: {before_shape[0]:,} rows x {before_shape[1]} columns")
print(f"Dataset dimensions AFTER:  {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"Records removed:           {before_shape[0] - df.shape[0]:,}")
print(f"Columns removed:           {before_shape[1] - df.shape[1]}")
print(f"Null cells remaining:      {df.isnull().sum().sum():,}")
print()
print("The cleaning pipeline is modular and reusable for future dataset updates.")
print("Each step can be independently configured or extended for different data sources.")

Dataset dimensions BEFORE: 125,000 rows x 13 columns
Dataset dimensions AFTER:  120,000 rows x 10 columns
Records removed:           5,000
Columns removed:           3
Null cells remaining:      78,257

The cleaning pipeline is modular and reusable for future dataset updates.
Each step can be independently configured or extended for different data sources.


---
## Section 5 — Export

In [23]:
output_file = "cleaned_ecommerce_data.csv"
df.to_csv(output_file, index=False)
print(f"Saved cleaned dataset to: {output_file}")
print(f"Row count: {len(df):,}")
print(f"Columns:   {list(df.columns)}")

Saved cleaned dataset to: cleaned_ecommerce_data.csv
Row count: 120,000
Columns:   ['order_id', 'sku', 'product_name', 'order_date', 'price', 'quantity', 'customer_email', 'customer_phone', 'shipping_country', 'status']
